# TP1 - 22.45 Redes Neuronales - Regresión Logística y Lineal

## Regresión Logística

### Import required libraries and dataset

In [ ]:
from keras.datasets import fashion_mnist
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import datetime
from os.path import exists

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

### Download and load Fashion MNIST dataset

In [ ]:
(train_X, train_y), (test_X, test_y) = fashion_mnist.load_data()

### Normalize the dataset

In [ ]:
data_max = np.max(train_X)
train_X = train_X.astype('float32') / data_max
test_X = test_X.astype('float32') / data_max
np.max(train_X)

### Exploratory Data Analysis

#### Example: Show the first object of the train dataset

In [ ]:
plt.imshow(train_X[0,...], cmap="gray")

#### Show a few instances of each class

In [ ]:
fig = plt.figure(figsize=(40, 40))  # width, height in inches

# idx works on np.array and not lists.
idx = np.argsort(train_y)

train_X_sorted = np.array(train_X)[idx]
train_y_sorted = np.array(train_y)[idx]

count = 0

for i in range(100):
    count = int(np.floor(i / 10))
    sub = fig.add_subplot(10, 10, i + 1, xticks=[], yticks=[])
    sub.imshow(train_X_sorted[i + count * 6000,:,:], interpolation='nearest', cmap='gray')
    sub.set_title('Category: ' + str(train_y_sorted[i + count * 6000]))

#### Look at the data distribution

##### Training data

In [ ]:
unique, counts = np.unique(train_y, return_counts=True)
print(dict(zip(unique, counts)))

counts = np.bincount(train_y)
print(counts)

fig, ax = plt.subplots(figsize=(10,5))
ax.bar(range(10), counts, width=0.8, align='center')
ax.set(xticks=range(10), xlim=[-1, 10], title='Training data distribution')

plt.show()

##### Testing data

In [ ]:
unique, counts = np.unique(test_y, return_counts=True)
print(dict(zip(unique, counts)))

counts = np.bincount(test_y)
print(counts)

fig, ax = plt.subplots(figsize=(10,5))
ax.bar(range(10), counts, width=0.8, align='center')
ax.set(xticks=range(10), xlim=[-1, 10], title='Testing data distribution')

plt.show()

### Convert the dataset from a vector form to a categorical distribution

In [ ]:
num_classes = np.max(train_y) + 1
train_y_cat = utils.to_categorical(train_y, num_classes)
test_y_cat = utils.to_categorical(test_y, num_classes)

### Softmax

#### Config the model to be trained

In [ ]:
#train_X = train_X.reshape(train_X.shape[0], 28, 28, 1).astype('float32')
#test_X = test_X.reshape(test_X.shape[0], 28, 28, 1).astype('float32')

In [ ]:
softmax_model =  Sequential()
#model.add(preprocessing.RandomFlip("horizontal", input_shape=(28,28,1)))
# model.add(layers.Dropout(0.1, input_shape=(28,28)))
softmax_model.add(layers.Flatten(input_shape=(28,28)))
softmax_model.add(layers.Dense(128, activation="relu"))
softmax_model.add(layers.Dense(num_classes, activation="softmax"))
softmax_model.summary()

#### Compile the model

In [ ]:
softmax_model.compile(loss = 'categorical_crossentropy', optimizer=SGD(learning_rate=0.0002, momentum=0.95),metrics=["accuracy"])

In [ ]:
# Callback to stop training if, after 2 epochs, the accuracy is not improving
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)

In [ ]:
# Callback to save the weights of the best model
checkpoint_filepath = '/tmp/checkpoint/softmax'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
# Logs and metrics from TensorBoard
log_dir = "logs/fit/softmax/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#### Fit the model to the train data and validate it with the test data

In [ ]:
# We load the previously best weights to save time on training
# if (exists(checkpoint_filepath)):
    # softmax_model.load_weights(checkpoint_filepath)

In [ ]:
softmax_history = softmax_model.fit(train_X, train_y_cat, validation_data=(test_X, test_y_cat), batch_size = 64, epochs=10, callbacks=[tensorboard_callback, early_stop_callback])
# softmax_history = softmax_model.fit(train_X, train_y_cat, validation_data=(test_X, test_y_cat), batch_size = 64, epochs=10, callbacks=[model_checkpoint_callback, tensorboard_callback, early_stop_callback])

#### Plot important metrics

##### TensorBoard session

In [ ]:
%tensorboard --logdir logs/fit

##### Loss

In [ ]:
plt.plot(softmax_history.history["loss"], label="train")
plt.plot(softmax_history.history["val_loss"], label="val")
plt.legend()
plt.show()

##### Accuracy

In [ ]:
plt.plot(softmax_history.history["accuracy"], label="train")
plt.plot(softmax_history.history["val_accuracy"], label="val")
plt.legend()
plt.show()

### MLP

In [ ]:
mlp_model = Sequential()
mlp_model.add(layers.Flatten(input_shape=(28,28)))
mlp_model.add(layers.Dense(256, activation='relu'))
mlp_model.add(layers.Dense(64, activation='relu'))
mlp_model.add(layers.Dense(num_classes, activation="softmax"))
mlp_model.summary()

#### Compile the model

In [ ]:
mlp_model.compile(loss = 'categorical_crossentropy', optimizer=SGD(learning_rate=0.0002, momentum=0.95),metrics=["accuracy"])

In [ ]:
# Callback to stop training if, after 2 epochs, the accuracy is not improving
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)

In [ ]:
# Callback to save the weights of the best model
checkpoint_filepath = '/tmp/checkpoint/mlp'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
# Logs and metrics from TensorBoard
log_dir = "logs/fit/mlp/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#### Fit the model to the train data and validate it with the test data

In [ ]:
# We load the previously best weights to save time on training
# if (exists(checkpoint_filepath)):
    # mlp_model.load_weights(checkpoint_filepath)

In [ ]:
mlp_history = mlp_model.fit(train_X, train_y_cat, validation_data=(test_X, test_y_cat), batch_size = 64, epochs=10, callbacks=[tensorboard_callback, early_stop_callback])
# softmax_history = softmax_model.fit(train_X, train_y_cat, validation_data=(test_X, test_y_cat), batch_size = 64, epochs=10, callbacks=[model_checkpoint_callback, tensorboard_callback, early_stop_callback])
# mlp_history = mlp_model.fit(train_X, train_y_cat, validation_data=(test_X, test_y_cat), batch_size = 64, epochs=10, callbacks=[model_checkpoint_callback, tensorboard_callback, early_stop_callback])

#### Plot important metrics

##### TensorBoard session

In [ ]:
%tensorboard --logdir logs/fit

##### Loss

In [ ]:
plt.plot(mlp_history.history["loss"], label="train")
plt.plot(mlp_history.history["val_loss"], label="val")
plt.legend()
plt.show()

##### Accuracy

In [ ]:
plt.plot(mlp_history.history["accuracy"], label="train")
plt.plot(mlp_history.history["val_accuracy"], label="val")
plt.legend()
plt.show()